# Credit Card Fraud Detection

In [ ]:
COLAB = True

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
if COLAB:
    id_ = "1uSFC-iy_-_NkK-AEjoPMJI6ximN-jM5F"
    aux = "'https://docs.google.com/uc?export=download&id={}&confirm=t'".format(id_)
    !wget $aux -O ./creditcard.csv.zip
    !unzip -qq ./creditcard.csv.zip

In [ ]:
!ls -la

In [ ]:
# here the import csv file as a pandas dataframe
df = pd.read_csv("./creditcard.csv")

In [ ]:
pd.options.display.max_columns = None
df.tail()

In [ ]:
df.var()

In [ ]:
df["Time"].min(), df["Time"].max()

In [ ]:
172792.0 / (60*60*24) # número de segundos en un día

**Dataset description**

The dataset contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation.
Due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [ ]:
df.isna().sum()

In [ ]:
print("df dimensions: {} rows by {} columns".format(df.shape[0], df.shape[1]))

In [ ]:
df.columns

In [ ]:
y = df['Class'].values
y.shape

In [ ]:
df['Time'].max() / 3600

In [ ]:
# now we drop the target variable from the data set
#df.drop(['Class', 'Time', 'Amount'],axis=1,inplace=True)
df.drop(['Class', 'Time'],axis=1,inplace=True)
#df.drop('Class',axis=1,inplace=True)
df.shape

In [ ]:
def show_counts(y):
    uns, counts = np.unique(y, return_counts=True)
    for u,c in zip(uns,counts):
        print("y={}: {} times".format(u,c))

In [ ]:
#distribution of the target variable

plt.figure(figsize=(5,4))
uns,counts = np.unique(y, return_counts=True)
plt.bar(uns,counts)
plt.xticks([0,1])
plt.show()

show_counts(y)

In [ ]:
df.hist(figsize=(12,24), bins=100, layout=(-1,3));

In [ ]:
df.describe()

In [ ]:
show_counts(y[df.Amount >= 50])

In [ ]:
y[df.Amount >= 50].mean()

In [ ]:
y[df.Amount >= 500].mean()

In [ ]:
y[df.Amount >= 1000].mean()

In [ ]:
df.describe().T

In [ ]:
y

In [ ]:
attribute_names = list(df.columns)
print(attribute_names)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df[y==0]["V1"], df[y==0]["V2"], alpha=0.1, label="Ok")
plt.scatter(df[y==1]["V1"], df[y==1]["V2"], alpha=0.1, label="Ko")
plt.legend();

In [ ]:
df["Amount"] = df["Amount"] / 125

In [ ]:
df.describe()

In [ ]:
from sklearn.model_selection import train_test_split

df_tr, df_te, y_tr, y_te = train_test_split(df, y, test_size=0.3, random_state=2, stratify=y)

In [ ]:
y_tr.mean(), y_te.mean()

In [ ]:
df_tr.shape

In [ ]:
# preparo conjunto de training y de validación
df_tr = df_tr[y_tr==0] # me quedo solo con los ejemplos ok
df_tr, df_val = train_test_split(df_tr, test_size=0.3, random_state=1) # el 30% lo llevo a validación

In [ ]:
df_tr.shape, df_val.shape

In [ ]:
# Isolation Forest:

from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=100, contamination="auto")
# contamination: tasa de outliers. Es “auto” o debe estar en (0, 0.5]

clf.fit(df_tr)
outliers_train = clf.predict(df_tr) # -1 si outlier, 1 si ok (inlier)
outliers_test  = clf.predict(df_te)

In [ ]:
len(y_te[outliers_test==-1])

In [ ]:
sum(y_te[outliers_test==-1])

In [ ]:
scores_train = -clf.decision_function(df_tr) # -1 si outlier, 1 si ok (inlier)
scores_test  = -clf.decision_function(df_te)

In [ ]:
scores_test

In [ ]:
np.unique(outliers_test)

In [ ]:
scores_test

In [ ]:
pd.options.display.max_rows = None
results_te = pd.DataFrame({"score_anomalía":scores_test, "clase":y_te})
results_te.sort_values("score_anomalía", ascending=False)[:500]

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpt, tpr, thresholds = roc_curve(y_true=y_te, y_score=scores_test, pos_label=1)

In [ ]:
y_te

In [ ]:
plt.plot(fpt, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AUC={}".format(roc_auc_score(y_true=y_te, y_score=scores_test)))
plt.grid()
#plt.xlim([0.01,0.1])
plt.show()
plt.plot(fpt, thresholds)
plt.xlabel("False Positive Rate")
plt.ylabel("Threshold")
#plt.xlim([0.01,0.1])
#plt.ylim([0,10])
plt.grid();

In [ ]:
len(y_te[scores_test>0.1])

In [ ]:
sum(y_te[scores_test>0.1])

In [ ]:
100*y_te.mean()

In [ ]:
umbral = 10
(y_te[scores_test>umbral] == 0).sum()

In [ ]:
(y_te == 0).sum()

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_te, scores_test, pos_label=1)

In [ ]:
plt.plot(recall, precision)
plt.xlabel("Recall (TPR)")
plt.ylabel("Precision")
plt.grid()
#plt.xlim([0.01,0.1])
plt.show()
plt.plot(recall[1:], thresholds)
plt.xlabel("Recall (TPR)")
plt.ylabel("Threshold")
#plt.xlim([0.01,0.1])
#plt.ylim([0,10])
plt.grid();